In [ ]:
from selenium import webdriver
import pandas as pd
from tqdm import tqdm
import time
from functools import reduce
import concurrent.futures

In [ ]:
filename = 'ndtv'
data = pd.read_csv(filename+'.csv')
print(data.shape)
data.head(7)

In [ ]:
degree_parellalism = 5
batch_size = 50
urllist = data["url"]

In [ ]:
def get_content(url_batch, pbar, tid = "mainThread"):
    driver = webdriver.Chrome(executable_path = '../utils/chromedriver')  # Ofline Mode
    # driver = webdriver.Chrome('chromedriver',options=chrome_options)    # Collab Mode
    title, content = [], []
    for url in url_batch[0]:
      if filename in url:
        try:
          driver.get(url)
          title.append(driver.find_element_by_xpath('/html/body/div[2]/div/div/section/div[2]/div[1]/h1').text)
          content.append(' '.join([item.text for item in driver.find_elements_by_xpath('//*[@id="ins_storybody"]/p')]))
        except:
          ;
      pbar.update(1)
    driver.close()
    return title, content

#Distribute regions for the multiple threads
def BatchGenerator(l, n):
    tid = 0
    for i in range(0, len(l), n):
        tid = tid + 1
        yield (l[i:i + n], 'Thread_'+str(tid))

def structure_data(content):
    title, body = list(map(list,zip(*content)))
    return [(title,body)]

In [ ]:
with tqdm(total=len(urllist)) as pbar:
    with concurrent.futures.ThreadPoolExecutor(max_workers=degree_parellalism) as executor:
        futures = [executor.submit(get_content, param, pbar) for param in BatchGenerator(urllist, batch_size)]
        return_value = [f.result() for f in futures]

In [ ]:
t = reduce(lambda a,b:((structure_data(a)).extend(structure_data(b))), return_value)
l = pd.DataFrame(t).T
l.columns = ["Title","Content"]
# l.to_csv(filename+'scraped.csv')
l

In [ ]:
driver = webdriver.Chrome(executable_path = '../utils/chromedriver')  # Ofline Mode
driver.close()
